In [40]:
import os

import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import boto3
import mlflow
from mlflow.models import infer_signature

In [41]:
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
mlflow.set_tracking_uri("http://127.0.0.1:5050")

In [46]:
mlflow.set_experiment("new_mlflow_artifact_registry_fixed")

2025/12/08 12:08:26 INFO mlflow.tracking.fluent: Experiment with name 'new_mlflow_artifact_registry_fixed' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-g2-new/2', creation_time=1765220906143, experiment_id='2', last_update_time=1765220906143, lifecycle_stage='active', name='new_mlflow_artifact_registry_fixed', tags={}>

In [47]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {"solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate accuracy as a target loss metric
accuracy = accuracy_score(y_test, y_pred)

/Users/theodorelowell/MIS547/G2_547_Final/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [48]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris-model",
        signature=signature,
        input_example=X_train,
        registered_model_name="basic_lr_iris_model",
    )

2025/12/08 12:08:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/08 12:08:38 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'basic_lr_iris_model'.
2025/12/08 12:08:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: basic_lr_iris_model, version 1
Created version '1' of model 'basic_lr_iris_model'.


🏃 View run respected-steed-920 at: http://127.0.0.1:5050/#/experiments/2/runs/761e443b87754a939d092a8d258fdcae
🧪 View experiment at: http://127.0.0.1:5050/#/experiments/2


In [49]:
from mlflow import MlflowClient

# Initialize an MLflow Client
client = MlflowClient()

def assign_alias_to_stage(model_name, stage, alias):
    """
    Assign an alias to the latest version of a registered model within a specified stage.

    :param model_name: The name of the registered model.
    :param stage: The stage of the model version for which the alias is to be assigned. Can be
                "Production", "Staging", "Archived", or "None".
    :param alias: The alias to assign to the model version.
    :return: None
    """
    latest_mv = client.get_latest_versions(model_name, stages=[stage])[0]
    client.set_registered_model_alias(model_name, alias, latest_mv.version)